# Ptolemy-Server Explanatory

Persistent server is recommended over cli for runtime and enables active learning. 

The server itself is essentially a lightweight fastapi wrapper over the Ptolemy and Ptolemy_AL classes. 

Required packages beyond base Ptolemy/Ptolemy_AL packages: fastapi, uvicorn, pydantic


### Running the server

navigate to the ptolemy directory (the directory in which this notebook is contained) and run 

```
uvicorn ptolemy.ptolemy_server:app --host [hostip] --port [port]
```

## Example Workflow

Initialize server, for this tutorial we will assume it's running locally at port 8000\
```baseurl = 'http://127.0.0.1:8000'```

New grid is loaded. Let's say we want to initialize a new session, with historical sessions.\
Make a post request to ```/initialize_new_session```. For example
```
payload={
    'historical_state_paths': [list of historical state paths]
}
payload = json.dumps(payload)
requests.post(baseurl + '/initialize_new_session', data=payload)
```
Next, the microscope operation software takes several low-mag tile images of the grid. After each tile image is taken, make a post request to ```/push_lm``` with the tile image. ```/push_lm``` automatically detects squares with the GMM, scores them with the prior model and adds their metadata to the ```current_lm_state```.

```
payload={
    'image': image.tolist(),
    'grid_id': # integer grid id
    'tile_id': # optional integer tile id
}
payload = json.dumps(payload)
requests.post(baseurl + '/push_lm', payload)
```
After all the tile images are pushed, if there are pre-selected squares from a screening session that are known to contain good holes, the microscope can be instructed to image these squares and holes. Alternatively, the server can immediately be queried via ```/select_next_square```, which runs the active learning GP on the unvisted squares in ```current_lm_state``` and returns the ```current_lm_state``` dataframe containing columns ```square_id, tile_id, grid_id, center, features, prior_score, visited, GP_probs```

They are as follows:
```
- square_id: a unique index for identifying the square
- tile_id: an integer identifier of the tile image that the square came from, None if not specified
- grid_id: an integer identifier for the grid that the square came from
- center: coordinates of the square centers
- features: vector of low-mag GP features: [mean_brightness, max_brightness, min_brightness, var_brightness, area, kurtosis_pixelintensities, skew_pixelintensities]
- prior_score: score as determined by the prior square classification model
- visited: boolean square has been visited/not visited
- GP_probs: Probability from GP that the square contains the most holes with CTF < 5
```
This information can be used to select a square to visit. For suggestions/recommendations based on your workflow, see TODO

[TODO Also should talk about setting the noice-hole-intensity]

Once a medium-mag image of the square has been taken, a post request to ```push_and_evaluate_mm``` with the image can be made:

```
payload={
    'image': image.tolist(),
    'square_id': # integer square id corresponding to the integer square id in current_lm_state
    'mm_img_id': # optional integer medium_mag image id
    'grid_id': integer grid id
}
payload = json.dumps(payload)
response = requests.post(baseurl + '/push_and_evaluate_mm', payload)
pd.read_csv(io.StringIO(response.json()), index_col='hole_id')
```

```push_and_evaluate_mm``` does the following:
- automatically detects holes in the medium-mag image
- scores the holes with the prior hole model
- adds the holes to the ```current_mm_state```
- sets the detected holes as the ```active_holes```, overwriting any holes that were active previously
- runs the medium-mag active learning GP on the ```active_holes```
- returns a ```hole_results``` dataframe containing the current_mm_state for the active_holes that can be used to decide which holes to image.

The hole results dataframe contains columns ```hole_id, mm_img_id, square_id, grid_id, center, features, prior_score, visited, radius, ctf, ice_thickness, ctf_pred, ice_pred, ctf_var, ice_var```\
They are as follows:
```
- hole_id: a unique index for identifying the hole
- mm_img_id: an optional integer identifier for the medium mag image that this hole is associated with
- square_id: an integer identifier for the square that this hole is in. Linked to the square_id in the current_lm_state.
- grid_id: an integer identifer for the grid that the hole is in. Linked to the grid_id in the current_lm_state
- center: coordinates of the hole centers
- features: vector of hole features extracted from the hole-feature extraction model, which are used by the medium-mag GP
- prior_score: score of hole as determined by the prior hole classification model
- visited: boolean hole has been visited/not visited
- radius: radius of the hole determined by the image processing algorithm. Currently they will be identical for all holes in a medium-mag image, but future functionality will allow for attempted refinement of hole radii. 
- ctf: true ctf resolution (Angstroms) of hole. This is set when a hole is visited, and will be None for holes that have not been visited
- ice_thickness: true ice thickness (nm) of hole. This is set when a hole is visited, and will be None for holes that have not been visited
- ctf_pred: mean of the posterior ctf distribution for the hole predicted by the GP
- ice_pred: mean of the posterior ice thickness distribution for the hole predicted by the GP
- ctf_var: variance of the posterior ctf distribution for the hole predicted by the GP
- ice_var: variance of the posterior ice thickness distribution for the hole predicted by the GP
```

This information can be used to select which holes to visit. For suggestions/recommendations based on your workflow, see TODO. In general, we believe that given the relatively low cost of imaging an additional hole at this stage (30 seconds - 1 minute) and the high cost of missing a good hole, we recommend only excluding holes that the model is very certain is bad. 

As holes are visited and ctf and ice thickness is collected, this information can be used to update the state using ```/visit_holes```. 

```
payload={
    'hole_ids': [list of visited hole ids]
    'ctfs': [list of floats of the ctf resolution of the holes]
    'ice_thicknesses': [list of floats of the ice thickness of the holes]
}
payload = json.dumps(payload)
requests.post(baseurl + '/visit_holes', payload)
```
This also automatically marks the square that contains the hole as visited, and removes the hole from the set of ```active_holes```. 

The GP can be rerun on unvisited holes in the medium-mag image at any point by calling the GET request, ```/rerun_mm_on_active_holes```. This returns the hole_results dataframe but with fresh GP predictions using all hole data that was added to the current_mm_state, while any visited holes were automatically removed from the ```active_holes``` by ```/visit_holes```. 

If necessary, the GP can be rerun on an arbitrary set of holes via ```rerun_mm_on_arbitrary_holes```. 

Once imaging in a square has been exhausted, the workflow can be repeated by acquiring next square data by again calling ```select_next_square```, which will run the GP on the unvisited squares. 

[Probably say something about switching grids]

Once collection on this session is finished, in order to clear the current session state and load potentially new historical and current states, call ```/initialize_new_session```.
```
payload={
    'new_state_path': optional path to the new current state to load
    'historical-state_paths': optional list of paths to historical states to be loaded
    'save_state_path': optional path to save the current state at. 
}
payload = json.dumps(payload)
requests.post(baseurl + '/initialize_new_session', payload)
```